In [ ]:
try:
    get_ipython().magic('load_ext autoreload')
    get_ipython().magic('autoreload 2')
except NameError:
    pass

from IPython.display import display, clear_output
import glob
import logging
import numpy as np
import os
import scipy
import cv2

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import caiman as cm
from caiman.source_extraction import cnmf as cnmf
from caiman.paths import caiman_datadir
from caiman.utils.utils import download_demo
import matplotlib.pyplot as plt

import bokeh.plotting as bpl
try:
       from bokeh.io import vform, hplot
except:
       # newer version of bokeh does not use vform & hplot, instead uses column & row
       from bokeh.layouts import column as vform
       from bokeh.layouts import row as hplot
from bokeh.models import CustomJS, ColumnDataSource, Slider

bpl.output_notebook()

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fnames` variable as a list. Note that the memory requirement of the CNMF-E algorithm are much higher compared to the standard CNMF algorithm. Test the limits of your system before trying to process very large amounts of data.

In [ ]:
fnames = ['data_endoscope.tif']  # filename to be processed
fnames = [download_demo(fnames[0])]
fnames = ['/Users/epnevmatikakis/Documents/Ca_datasets/Miniscope/msCam13.avi']

In [ ]:
# dataset dependent parameters
fr = 10                          # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True            # flag for performing motion correction
pw_rigid = False                 # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (7, 7)               # size of high pass spatial filtering, used in 1p data
max_shifts = (15, 15)            # maximum allowed rigid shift
strides = (96, 96)               # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)              # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 5          # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'              # replicate values along the boundaries

mc_dict = {
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}


rf = 48
stride = 8
ssub = 1
ds_factor = 4*ssub

gSig = (12//ds_factor, 12//ds_factor)            # expected half size of neurons
gSiz = (36//ds_factor, 36//ds_factor)
p = 0                                          # order of AR indicator dynamics
min_SNR = 1.5                                  # minimum SNR for accepting new components
min_pnr = 10                                   # minimum PNR during cnmf-E initialization
min_corr = 0.8                                 # minimum corr image value during cnmf-E initialization
rval_thr = 0.85                                # correlation threshold for new component inclusion
sniper_mode = False                            # flag using a CNN to detect new neurons (o/w space correlation is used)
init_batch = 300                               # number of frames for initialization (presumably from the first file)
expected_comps = 500                           # maximum number of expected components used for memory pre-allocation (exaggerate here)
dist_shape_update = False                      # flag for updating shapes in a distributed way
min_num_trial = 8                              # number of candidate components per frame     
K = None                                       # initial number of components
epochs = 2                                     # number of passes over the data
show_movie = False                             # show the movie with the results as the data gets processed

cnmfe_dict = {'fnames': fnames,
              'fr': fr,
              'decay_time': decay_time,
              'method_init': 'corr_pnr',
              'gSig': gSig,
              'gSiz': gSiz,
              'rf': rf,
              'stride': stride,
              'p': p,
              'epochs': 2,
              'nb': 0,
              'ssub': ssub,
              'ds_factor': ds_factor,                                   # ds_factor >= ssub should hold
              'min_SNR': min_SNR,
              'min_pnr': min_pnr,
              'min_corr': min_corr,
              'bas_nonneg': False,
              'center_psf': True,
              'rval_thr': rval_thr,
              'init_batch': init_batch,
              'only_init': True,
              'init_method': 'cnmf',
              'normalize_init': False,
              'update_freq': 200,
              'expected_comps': expected_comps,
              'sniper_mode': sniper_mode,
              'dist_shape_update' : dist_shape_update,
              'min_num_trial': min_num_trial,
              'merge_thr': 0.65,
              'K': K,
              'epochs': epochs,
              'show_movie': show_movie}
opts = cnmf.params.CNMFParams(gnb=0, params_dict={**mc_dict, **cnmfe_dict})

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
gSiz

In [ ]:
cnm = cnmf.online_cnmf.OnACID(params=opts, dview=dview)
cnm.fit_online()

In [ ]:
images = cm.load(fnames[0], subindices=slice(0,1000))
Cn, pnr = cm.summary_images.correlation_pnr(images[::1], gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
cnm.estimates.nb_view_components(img=Cn, denoised_color='red');

In [ ]:
cnm.estimates.plot_contours_nb(img=Cn)

In [ ]:
cnm.estimates.plot_contours(img=Cn)

## Plot timing
The plot below shows the time spent on each part of the algorithm (motion correction, tracking of current components, detect new components, update shapes) for each frame. Note that if you displayed a movie while processing the data (`show_movie=True`) the time required to generate this movie will be included here.

In [ ]:
T_motion = 1e3*np.array(cnm.t_motion)
T_detect = 1e3*np.array(cnm.t_detect)
T_shapes = 1e3*np.array(cnm.t_shapes)
T_online = 1e3*np.array(cnm.t_online) - T_motion - T_detect - T_shapes
plt.figure()
plt.stackplot(np.arange(len(T_motion)), T_motion, T_online, T_detect, T_shapes)
plt.legend(labels=['motion', 'process', 'detect', 'shapes'], loc=2)
plt.title('Processing time allocation')
plt.xlabel('Frame #')
plt.ylabel('Processing time [ms]')
plt.ylim([0, 1.2e3*np.percentile(np.array(cnm.t_online), 90)]);

In [ ]:
cnm.estimates.play_movie(images, magnification=1, q_max=99.5, q_min=0.5, include_bck=True)